# TP4 MPI et les communications collectives
## M1 informatique, Université d'Orléans 2021/2022

L'objectif de ce TP est de mettre en place des programmes encore simples avec les routines de communications collectives. 
Vous allez également pouvoir exécuter vos codes sur une machine parallèle afin de faire quelques tests de performances.

### 1. Le calcul de $\pi$
La valeur de $\pi$ peut être calculée par l'intégrale $\int_{0}^{1} \frac{4}{1+x^2}$. Cette intégrale peut être approchée par $\pi \approx \sum_{i=0}^nf(x_i)\Delta x$ où $\Delta x=\frac{1}{n}$ et $x_i=\frac{1}{2n}+i\frac{1}{n}$

Plus $n$ est grand plus on se rapproche de $\pi$. 

In [ ]:
#include <mpi.h>
#include <iostream>

using namespace std;

double fx(double x)
{
  double res =(double)4.0/(1+x*x);
  return res;
}


int main ( int argc , char **argv )
{
  int pid, nprocs;  
  MPI_Init (&argc , &argv) ;
  MPI_Comm_rank(MPI_COMM_WORLD, &pid ) ;
  MPI_Comm_size (MPI_COMM_WORLD, &nprocs ) ;
  
  int n = atoi(argv[1]);
  int root = atoi(argv[2]);  
  
  double Pi;  
 
  // A compléter   
    
  if (pid==root) {
    cout << "PI=" << setprecision (15) << Pi << endl;
  }
  MPI_Finalize() ;
  return 0 ;
}


### 2. Normalisation d'un vecteur

Soit *V* un vecteur de taille *n* généré sur un processus *root*. On souhaite calculer la normalisation de ce vecteur en partageant le travail sur les différents processus et en rassemblant le résultat sur le processus *root*.

$$
V_{\operatorname{norm}} = \frac{1}{\left\|V\right\|} V \quad\mbox{avec}\quad
\left\|V\right\| = \sqrt{\sum_{i=0}^{n-1} V_i^2}
$$

In [ ]:
Q3. Modifiez la version précédente pour lever la condition n divisible par le nombre de processus.

### 3. Exécution sur ptimirev

La machine *ptimirev* est une grappe de PC reliée par un réseau Ethernet. Elle est constituée de 5 nœuds dont ptimirev-server qui est le point d'entrée à partir de votre compte des salles machines (adresse IP de ptimirev-server 192.168.80.201). Ensuite vous pouvez accéder aux 4 autres nœuds ptimirev1, ptimirev2, ptimirev3 et ptimirev4.


3.0 __Préliminaires__
    1. ssh o'num étudiant'@ptimirev-server (mot de passe code NNE)
    2. ssh-keygen (tout valider sans rentrer de mot de passe)
    3. cat .ssh/id_rsa.pub >> .ssh/authorized_keys (pour ajouter la clé publique aux autorisations)
    4. for i in 1 2 3 4; do ssh ptimirev$i echo Ok; done (valider l'identité des 4 machines)
    5. félicitations ! vous pouvez désormais vous connecter sans mot de passe aux 4 nœuds
    
Si vous préférez utiliser une clé SSH avec mot de passe, alors il est nécessaire avant de lancer MPI d'activer un agent d'authentification SSH et de lui fournir la clé privée. Cela peut se faire, par exemple, en une commande qu'il faudra utiliser à **chaque session** : eval $(ssh-agent); ssh-add

3.1 __Gestion de l'exécution parallèle__

Lorsqu'on exécute le programme sur une machine à mémoire distribuée, **mpirun** gère le lancement du programme sur chaque processus à distance par ssh. Il faut donc que l'exécutable soit accessible via le **PATH**. Pour cela vous pouvez rajouter au fichier **.bashrc** les lignes suivantes


In [ ]:
if [ -d "$HOME/bin" ] ; then
    export PATH="$HOME/bin:$PATH"
fi

Ainsi vous pourrez placer votre exécutable dans ce répertoire bin de votre $HOME et lors de l'exécution parallèle le lancement à distance par mpirun pourra fonctionner.

Enfin, avant de pouvoir exécuter le programme il faut définir les processus que l'on souhaite utiliser. Pour cela il faut créer un fichier qui va contenir ces informations. Voici un exemple (le fichier créé se nomme liste_machines)

ptimirev1 slots=2<br/>
ptimirev2 slots=2<br/>
ptimirev3 slots=2<br/>
ptimirev4 slots=2<br/>

La syntaxe pour utiliser ce fichier et lancer le programme sur les processus correspondants est

mpirun --hostfile liste_machines (-np 4) 'NomExecutable' 'les arguments'

Si vous précisez le nombre de processus avec l'option -np x le programme s'exécutera sur x processus choisis dans le fichier listes_machines. Sans cette option il utilise toutes les machines données dans le fichier.


3.2 __Performances de la normalisation d'un vecteur__

In [ ]:
#include <iostream>
#include <random>
#include <chrono>
#include <mpi.h>

using namespace std;

int main(int argc, char **argv) {
    int pid, nprocs;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &pid);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

    chrono::time_point<chrono::system_clock> start, end;
    
    int n = atoi(argv[1]); // la taille du tableau global
    int root = atoi(argv[2]); // le processeur root
    
    double *vecteur;

    if (pid == root) {
      vecteur = new double[n];
      random_device rd;
      mt19937 gen(rd());
      uniform_real_distribution<> dis(0, 10.0);
      for (int i = 0; i < n; i++)
        vecteur[i] = dis(gen);
      
      cout << "le tableau initial est ";
      for (int i = 0; i < n; i++)
        cout << vecteur[i] << " ";
      cout << endl;
    }

    if (pid==root) {
      start = chrono::system_clock::now();
    }

    // Votre code ici
    
    if (pid==root) { //Calcul du temps écoulé.
      end = chrono::system_clock::now();
      chrono::duration<double> elapsed_seconds = end-start;
      cout << "tps=" << elapsed_seconds.count() << endl;
    }
    
    MPI_Finalize();
    return 0;
}

### 4. Suite monotone

Une suite est monotone si elle est strictement croissante ou strictement décroissante. On souhaite définir si un tableau de réels $U$ de taille $n$ correspond à une suite monotone. Le processus *root* est le seul à avoir initialement le tableau $U$.

Dans une première étape le tableau $U$ est distribué avec un *MPI_Scatterv*. 
